# Convulational Neural Network Assignment
The goal of this project is to train a regression CNN to predict the resolution. We
will use the square loss functions on the training examples $(\mathbf{x}_i, y_i),i=1, \dots,n$:

$$S(\mathbf{w})=\frac{1}{n}\Sigma_{i=1}^n(y_i-f_\mathbf{w}(\mathbf{x}_i))^2 + \lambda ||\mathbf{w}||^2$$

Besides the loss function, we will measure the $R^2$

$$R^2=1-\frac{\Sigma_{i=1}^n(y_i-\hat{y}_i)^2}{\Sigma_{i=1}^n(y_i-\bar{y}_i)^2}$$

where $\hat{y}=f_\mathbf{w}(\mathbf{x}_i)$ and $\bar{y}=\frac{1}{n}\Sigma_{i=1}^n y_i$.

Experiment with different CNN architectures to obtain a good result. One example of a CNN you could use contains five convolutional layers with stride $1$ and zero
padding, the first four with filters of size $5 \times 5$ with or without holes (atrous), and the
last of the appropriate size to obtain a $1 \times 1$ output. The first two convolutions have
$16$ filters, the next two have $32$ filters, and the last has one filter. The first three convolutions are followed by $2 \times 2$ max pooling with stride $2$ respectively. The fourth
convolution layer is followed by ReLU.


#### Import dependencies

In [300]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from scipy.special import expit
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import MeanSquaredError
from keras.regularizers import l2
from keras.layers import Input
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers import Conv2D
from keras import Model
from keras import backend as K
import os

#### Data loading function

In [301]:

def get_data():
    data_path = 'data/cnn'
    train_path = os.path.join(data_path,'cnntrain')
    test_path = os.path.join(data_path,'cnntest')

    data_gen_train = ImageDataGenerator()
    train_dataframe = pd.DataFrame(
        zip([os.path.join(train_path, image) for image in os.listdir(train_path)],
        [int(filename[:2])/100 for filename in os.listdir(train_path)]),
        columns=['path', 'resolution'])
    image_gen_train= data_gen_train.flow_from_dataframe(train_dataframe,
        x_col='path', y_col='resolution', class_mode='raw', target_size=(64,64))

    data_gen_test = ImageDataGenerator()
    test_dataframe = pd.DataFrame(
        zip([os.path.join(test_path, image) for image in os.listdir(test_path)],
        [int(filename[:2])/100 for filename in os.listdir(test_path)]),
        columns=['path', 'resolution'])
    image_gen_test= data_gen_test.flow_from_dataframe(test_dataframe,
        x_col='path', y_col='resolution', class_mode='raw', target_size=(64,64))

    def normalized_image_gen(image_gen):
        while True:
            batch, labels = next(image_gen)
            batch /= 255
            #batch = np.split(batch, 3, axis=3)
            #batch = [np.squeeze(color) for color in batch]
            #batch = np.transpose(batch, (3, 0, 1, 2))
            #batch = [b for b in batch]
            yield batch, labels
    return normalized_image_gen(image_gen_train), normalized_image_gen(image_gen_test), train_dataframe.shape[0] // 32, test_dataframe.shape[0]

#### $R^2$ function

In [302]:
def R2(y_true, y_predicted):
    return 1 - K.sum((y_true-y_predicted)**2) / (K.sum((y_true-K.mean(y_true))**2)+K.epsilon())

#### Part A
Train a CNN for $100$ epochs with momentum $0.9$ using the square loss (1). Use
the SGD optimizer with an appropriate learning rate and $\lambda = 0.0001$ (weight
decay). Start with minibatch size $32$ and double it every $20$ epochs and to obtain
a good training $R^2$
(at least $0.9$). Show a plot of the loss function vs epoch
number for the training set and the test set. Show another plot of the training and
test $R^2$ vs epoch number. (4 points)

In [303]:
def build_cnn():
    #images, labels = next(train)
    #flattened_images = np.reshape(images, (images.shape[0], 4096*3))
    lmbda = 0.0001
    input_shape = (64, 64, 3)

    filters = (4,8)

    input = Input(shape=input_shape)
    x = input
    for f in filters:
        x = Conv2D(f, (3,3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dense(4)(x)
    x = Activation("relu")(x)
    x = Dense(1, activation="linear", kernel_regularizer=l2(lmbda))(x)

    model = Model(input, x)
    model.compile(optimizer="SGD", loss=MeanSquaredError(), metrics=[R2])

    #red_input = Input(shape=(64,64))
    #red_network = Flatten()(red_input)
    #red_network = Dense(8, activation="relu")(red_network)
    #red_network = Dense(4, activation="relu")(red_network)
    #red_network = Model(inputs=red_input, outputs=red_network)

    #green_input = Input(shape=(64,64))
    #green_network = Flatten()(green_input)
    #green_network = Dense(8, activation="relu")(green_network)
    #green_network = Dense(4, activation="relu")(green_network)
    #green_network = Model(inputs=green_input, outputs=green_network)

    #blue_input = Input(shape=(64,64))
    #blue_network = Flatten()(blue_input)
    #blue_network = Dense(8, activation="relu")(blue_network)
    #blue_network = Dense(4, activation="relu")(blue_network)
    #blue_network = Model(inputs=blue_input, outputs=blue_network)

    #merged_network = concatenate([red_network.output, green_network.output, blue_network.output])
    #merged_network = Dense(4, activation="relu")(merged_network)
    #merged_network = Dense(1, activation="linear", kernel_regularizer=l2(lmbda))(merged_network)

    #model = Model(inputs=[red_network.input, green_network.input, blue_network.input], outputs=merged_network)
    #model.compile(optimizer="SGD", loss=MeanSquaredError(), metrics=[R2])
    return model

def train_cnn(model, train, test, train_steps, test_steps):
    #print(model.summary())
    model.fit(train, steps_per_epoch=train_steps)
    print(model.evaluate(test, steps=test_steps, use_multiprocessing=False))

train, test, train_steps, test_steps = get_data()
#train_steps, test_steps = 50,50 # only train on some of the data to make testing faster
print("train_steps", train_steps)
print("test_steps", test_steps)
model= build_cnn()
train_cnn(model, train, test, train_steps, test_steps)

Found 18059 validated image filenames.
Found 2261 validated image filenames.
train_steps 564
test_steps 2261
Epoch 1/1
2261/2261 [==============================] - 35s 16ms/step